In [1]:
"""This is video reader with all operations inside fnctions, so they can be called on 
    each frame out of a set of frames, and each video out of a set of videos
    """

'This is video reader with all operations inside fnctions, so they can be called on \n    each frame out of a set of frames, and each video out of a set of videos\n    '

In [2]:
"""Imports"""
import matplotlib.pyplot as plt #for maknig plots inside the notebook
import skimage
import skvideo.io
from skvideo.io import vread
from skimage import measure
from skimage.filters import threshold_yen, threshold_isodata
from operator import attrgetter
import numpy as np
import math
from itertools import chain
#from functools import partial, update_wrapper
from scipy import ndimage 
from pims import pipeline, Video
from glob import glob
import seaborn as sns
#import antigravity

In [11]:
"""Functions"""
# apply total thresholding to each of a list of frames; filter slides with wrong number
def total_threshold_filter(framelist, trans_fil): 
    # returns a new list, different from the orginal
    fll = len(framelist)
    m_lab = measure.label
    m_rop = measure.regionprops
    
    sides = {"bm":[], "b":[], "t":[], "ucf":[]}
    classes = {"bm":[], "br":[], "bl":[], "tr":[], "tl":[], "ucf":[], "newt":[]}
    transitions = {"bm->bm" :[], "bm->br":[], "bm->bl":[], "bm->tr":[], "bm->tl":[],
                  "br->bm" :[], "br->br":[], "br->bl":[], "br->tr":[], "br->tl":[],
                  "bl->bm" :[], "bl->br":[], "bl->bl":[], "bl->tr":[], "bl->tl":[],
                  "tr->bm" :[], "tr->br":[], "tr->bl":[], "tr->tr":[], "tr->tl":[],
                  "tl->bm" :[], "tl->br":[], "tl->bl":[], "tl->tr":[], "tl->tl":[]}
    
    transition_threshold = trans_fil
    broken = 0
    broken_count = 0
    area_threshold = 1800   # must have enough particles
    convex_thresholds = (2000, 2800)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram
    minor_thresholds = (38, 48) # minor_axis length (< filterd) # particles must be in parallelogram
    major_thresholds = (59, 76)
    last_class = ''
    last_whole = 0
    filtrate_len = 0
    
    for i in range(fll):
       # print("new_frame")
        frame = framelist[i]
        thresh_img = frame > threshold_yen(frame) # binary image
        img_labelled = m_lab(thresh_img) # contains connected regions
        properties_list = m_rop(img_labelled, coordinates = 'rc') # data about regions, for each connected region
        
        #----------getting maximum connected region----------
        biggest_r = max(properties_list, key = attrgetter('area'))
        """ for k in range(len(properties_list)):
            if areas[k] > biggest_area:
                biggest_r = properties_list[k]
            else:
                pass"""

        #----------------------filter------------------------- can use actual filter
        test = (biggest_r.filled_area > area_threshold and   # must have enough particles
            biggest_r.minor_axis_length > minor_thresholds[0] # minor_axis length (< filterd) # particles must be in parallelogram
            and biggest_r.minor_axis_length < minor_thresholds[1]  # minor_axis length (< filterd) # particles must be in parallelogram
            and biggest_r.convex_area > convex_thresholds[0]  # convex hull area (> 2500 filtered)  # particles must be in parallelogram
            and biggest_r.convex_area < convex_thresholds[1]  # convex hull area (> 2500 filtered)  # particles must be in parallelogram
            and biggest_r.major_axis_length > major_thresholds[0]  # major_axis length (< filterd) # particles must be in parallelogram
            and biggest_r.major_axis_length < major_thresholds[1])  # major_axis length (< filterd) # particles must be in parallelogram
        if test: # keep frames that have enough paricles, and are not transitions
            i_threshed = frame > threshold_isodata(frame)
            img2_labelled = m_lab(i_threshed)
            properties2_list = m_rop(img2_labelled, coordinates = 'rc')
            biggest_r2 = max(properties2_list, key = attrgetter('area'))
            #-----------------------------classify pt 1---------------------------------------
            coprod = coord_prod(thresh_img, i_threshed)
            side = sideify(biggest_r2)
            clas = classify(side, -coprod)
            sides[side].append(i)
            classes[clas].append(i)
            if broken_count >= transition_threshold and side != 'ucf' and last_class not in ['', 'ucf', 'newt']: #if previous n frames were broken, and a transition between identifiableclasses ocurred
                transitions[last_class + "->" + clas].append((last_whole, i))
            broken = 0 #reset marker to not broken
            broken_count = 0
            last_class = clas
            last_whole = i
            filtrate_len += 1
        else:
            broken = 1 #set marker to broken
            broken_count += 1

    return [filtrate_len, sides, classes, transitions]



        
# take largest isodata image region, and return aclassification
def sideify(idir):
    iner = idir.inertia_tensor[0, 0] + idir.inertia_tensor[1, 1]
    maal = idir.major_axis_length
    mial = idir.minor_axis_length
    sol = idir.solidity 
    ecc = idir.eccentricity
    cva = idir.convex_area
    if sol <= 0.725 or (iner> 405 and mial > 43.5): # sol <.72
        return "bm"
    elif ecc > 0.8 or maal > 65: # or (mial < 41 and maal > 65): # it's on top #ec >.82
        return "t"
    elif ecc < 0.75 or maal < 64.5: #or mial > 42  #and cva > 2200 and iner < 325: #or (maal < 64 and iner < 360): # it's on bottom
        return "b"
    else:
        return "ucf"

def classify(side, coprod):
    if side == "t":
        if coprod > 0:
            return "tl"
        else:
            return "tr"
    elif side == "b": 
        if coprod > 0:
            return "br"
        else:
            return "bl"
    elif side == "bm":
        return "bm"
    elif side == "ucf":
        return "ucf"
    else:
        return "newt"


def coord_prod(thresh_y, thresh_i):
    m_rop = measure.regionprops
    for i in range(0, 1):#len(fl):
        full = largest_region_extractor(measure.label(thresh_y))
        four = largest_region_extractor(measure.label(thresh_i))
        full_or = m_rop(full.astype(int), coordinates = 'rc')[0].orientation
        one_and_some = full^four
        one = largest_region_extractor(measure.label(one_and_some))
        
          
        #Begin finding substitue for major and minor axes herre
        # use min and maax x and  values to find the geometric center
        # or use those to substitue for eigenvectors 
        rotated_one = ndimage.rotate(one, math.degrees(-full_or), reshape = False)
        rotated_full = ndimage.rotate(full, math.degrees(-full_or), reshape = False)
        geo_cent = m_rop(rotated_full.astype(int), coordinates = 'rc')[0].bbox
        #print(geo_cent)
        ave = np.average
        cent_x = ave([geo_cent[1], geo_cent[3]]) 
        cent_y = ave([geo_cent[0], geo_cent[2]])
        #print(cent_x, cent_y)
        
        raw_coords = (m_rop(rotated_full.astype(int), coordinates = 'rc')[0].coords).T
        ys = raw_coords[0]
        xs = raw_coords[1]
        x = xs - cent_x
        y = ys - cent_y      
        
        one_com = m_rop(rotated_one.astype(int))[0].centroid
        
        rel_com_one = (one_com[1] - cent_x, one_com[0] - cent_y)
        product = np.prod(rel_com_one)
        return product
        
def rotate_point(point, angle):
    x0 = point[0]
    y0 = point[1]
    x1 = x0*np.cos(angle) - y0*np.sin(angle)
    y1 = x0*np.sin(angle) + y0*np.cos(angle)
    return (x1, y1)
 
def region_selector(labeled_image, label):
    #print(label)
    x = labeled_image == label
    #plt.imshow(x)
    return x

# make a function that takes a set of labeled regions, and then  returns a boolean array containing only the largest
def largest_region_extractor(labeled_regions_set):
    props_lists = measure.regionprops(labeled_regions_set)
    #print(len(labeled_regions_set), len(props_lists))
    biggest_r_p = props_lists[0]
    biggest_r_label = props_lists[0].label
    for i in range(0, len(props_lists)):
        pli = props_lists[i]
        if pli.area > biggest_r_p.area:
            biggest_r_p = pli
            biggest_r_label = pli.label
        else:
            pass
    return region_selector(labeled_regions_set, biggest_r_label)    
#from pair of labelled, thresholded images, return an image category (of three)

#from pair of labelled,thresholded images, return the x/y sign or x/y magnitude for the transparent dot


In [12]:
"""Pipeline for lists of frames"""
""""""
"""Pipeline for videos:"""
# initialize frame lists
filenames = glob("./tchakamau/New_setup/*[0-9].avi")#glob("./tchakamau/New_setup/*[0-9]*filtered.avi") + glob("./tchakamau/New_setup/*23.avi")#["./tchakamau/New_setup/45681_10Vpp_25.avi", "./tchakamau/New_setup/45701_10Vpp_20.avi"]#glob("./tchakamau/New_setup/*.avi")#  #filenames = 
print(filenames)
num_top_keys = 4
num_full_keys = 7
num_tran_keys = 5
Top_bottoms = np.zeros((len(filenames), num_top_keys))
Full_classif = np.zeros((len(filenames), num_full_keys))
N_transitions = np.zeros((len(filenames), num_tran_keys, num_tran_keys))
trans_fil = 10

ave = np.average
num_total_frames = 0    
print("Num files: ", len(filenames))
for vidnum in range(len(filenames)):
    frames = vread(filenames[vidnum])
    print("read vid %s" %vidnum)
    frame_vid = frames[:, :, :, 2]# making videos ino a frame list
    num_total_frames += len(frame_vid)
    print("processing vid %s" %vidnum)
    org = total_threshold_filter(frame_vid, trans_fil)
    filtered_len = org[0]
    sids = org[1]
    cls = org[2]
    tran = org[3]
    #----------------------------------------------------------------
    trans_len = len([y for x in tran.values() for y in x])
    skeys = list(sids.keys())
    ckeys = list(cls.keys())
    tkeys = list(tran.keys())
    slcs = [len(sids[x]) for x in skeys]
    clcs = [len(cls[x]) for x in ckeys]
    trans = [len(tran[x]) for x in tkeys]
    ucf = len(sids["ucf"])
    newtucf = len(cls["newt"]) + ucf



    Top_bottoms[vidnum,:] = [slcs[i] for i in range(len(skeys))]
    Full_classif[vidnum,:] = [clcs[i] for i in range(len(ckeys))]
    N_transitions[vidnum,:] = np.reshape(trans, (num_tran_keys, num_tran_keys))
    print("finished vid %s" %vidnum)


    
#-----------------------------------------------------------------------




['./tchakamau/New_setup\\45621_10Vpp_22.avi', './tchakamau/New_setup\\45621_10Vpp_23.avi', './tchakamau/New_setup\\45681_10Vpp_24.avi', './tchakamau/New_setup\\45681_10Vpp_25.avi', './tchakamau/New_setup\\45681_10Vpp_26.avi', './tchakamau/New_setup\\45681_10Vpp_27.avi', './tchakamau/New_setup\\45701_10Vpp_18.avi', './tchakamau/New_setup\\45701_10Vpp_19.avi', './tchakamau/New_setup\\45701_10Vpp_20.avi', './tchakamau/New_setup\\45701_10Vpp_21.avi']
Num files:  10
read vid 0
processing vid 0
finished vid 0
read vid 1
processing vid 1
finished vid 1
read vid 2
processing vid 2
finished vid 2
read vid 3
processing vid 3
finished vid 3
read vid 4
processing vid 4
finished vid 4
read vid 5
processing vid 5
finished vid 5
read vid 6
processing vid 6
finished vid 6
read vid 7
processing vid 7
finished vid 7
read vid 8
processing vid 8
finished vid 8
read vid 9
processing vid 9
finished vid 9


In [5]:
print(N_transitions[0:5])

[[[11.  1.  1.  2.  8.]
  [ 4. 18. 15.  4.  3.]
  [ 2. 14. 15.  6.  5.]
  [ 7.  2. 10.  3.  5.]
  [ 4.  7.  5.  2.  2.]]

 [[ 2.  2.  0.  1.  0.]
  [ 2.  7.  4.  0.  0.]
  [ 3.  5.  6.  5.  2.]
  [ 3.  2.  7.  2.  2.]
  [ 1.  1.  2.  2.  3.]]

 [[ 3.  0.  1.  2.  0.]
  [ 3.  1.  0.  1.  2.]
  [ 0.  0.  3.  0.  1.]
  [ 4.  0.  0.  5.  3.]
  [ 4.  0.  3.  0.  1.]]

 [[ 2.  0.  0.  0.  2.]
  [ 1.  1.  1.  1.  0.]
  [ 0.  3.  2.  2.  1.]
  [ 2.  0.  0.  3.  0.]
  [ 0.  1.  2.  2.  1.]]

 [[ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]]]


In [13]:

"""Print average values (wuth column labels from slcs, clcs, trans) for rows of the array"""
var = np.var
sums = np.sum
Tb_sums = [sums(Top_bottoms[:, c]) for c in range(num_top_keys)]
Fc_sums = [sums(Full_classif[:, c]) for c in range(num_full_keys)]
Nt_sums = sums(N_transitions, axis = 0)#[sums(N_transitions[:, c]) for c in range(num_tran_keys)]
print("\nNumber of frames obbserved: ", num_total_frames)

Tb_percs = [Tb_sums[c] *100 /sum(Tb_sums) for c in range(num_top_keys)]
Fc_percs = [Fc_sums[c] *100/sum(Fc_sums) for c in range(num_full_keys)]
Nt_percs = Nt_sums *100/sums(Nt_sums)
#print(sums(N_transitions), sums(Nt_sums))


print("\nsums in broad classifications")
[print(skeys[i], "sum %.f" %Tb_sums[i]) for i in range(len(skeys))]
print("Total: ", sum(Tb_sums) )
print("\nsums in narrow classifications")
[print(ckeys[i], "sum %.f" %Fc_sums[i]) for i in range(len(ckeys))]
print("Total: ", sum(Fc_sums ) )
print("\nsums of transitions")
print(Nt_sums)
print("Total: ", sums(Nt_sums) )



print("\n%s in broad classifications")
[print(skeys[i], " %.f%%" %Tb_percs[i]) for i in range(len(skeys))]
print("\n%s in narrow classifications")
[print(ckeys[i], " %.f%%" %Fc_percs[i]) for i in range(len(ckeys))]
print("Transition matrix, broad")
T_eb = np.vstack([Nt_percs[0], 
                             (Nt_percs[1] + Nt_percs[2]) , 
                             (Nt_percs[3] + Nt_percs[4])])
Nt_percs_broad = np.column_stack([T_eb[:, 0], 
                             (T_eb[:, 1]+ T_eb[:, 2]), 
                             (T_eb[:, 3]+ T_eb[:, 4])])
print(Nt_percs_broad)
print("row_sums of transitin matrix, broad")
print(sums(Nt_percs_broad, axis = 1))
print("\n%s of transitions broad:")
print("bm->bm %.f%%" %Nt_percs[0, 0])
print("b->bm %.f%%" %sums(Nt_percs[1:3, 0]))
print("t->bm %.f%%" %sums(Nt_percs[3:5, 0]))
print("bm->b %.f%%" %sums(Nt_percs[0, 1:3]))
print("bm->t %.f%%" %sums(Nt_percs[0, 3:5]))
[print(skeys[int(np.ceil(i/2))]+"->"+ skeys[int(np.ceil(j/2))], "%.f%%" %(Nt_percs[i,j] + Nt_percs[i,j+1] + Nt_percs[i+1, j] + Nt_percs[i+1, j+1])) 
 for i in [1,3] for j in [1,3]]
print("\nTransition matrix, narrow")
print(Nt_percs)
print(sums(Nt_percs))
print("\n%s of transitions narrow")
[print(tkeys[i], " %.f%%" %Nt_percs.flatten()[i]) for i in range(len(tkeys))]



Number of frames obbserved:  493015

sums in broad classifications
bm sum 70883
b sum 120079
t sum 176777
ucf sum 1855
Total:  369594.0

sums in narrow classifications
bm sum 70883
br sum 78820
bl sum 41259
tr sum 86994
tl sum 89783
ucf sum 1855
newt sum 0
Total:  369594.0

sums of transitions
[[86. 14. 15. 30. 32.]
 [39. 75. 71. 39. 21.]
 [23. 64. 90. 34. 33.]
 [49. 22. 41. 46. 42.]
 [44. 29. 31. 35. 55.]]
Total:  1060.0

%s in broad classifications
bm  19%
b  32%
t  48%
ucf  1%

%s in narrow classifications
bm  19%
br  21%
bl  11%
tr  24%
tl  24%
ucf  1%
newt  0%
Transition matrix, broad
[[ 8.  3.  6.]
 [ 6. 28. 12.]
 [ 9. 12. 17.]]
row_sums of transitin matrix, broad
[16.69811321 46.13207547 37.16981132]

%s of transitions broad:
bm->bm 8%
b->bm 6%
t->bm 9%
bm->b 3%
bm->t 6%
b->b 28%
b->t 12%
t->b 12%
t->t 17%

Transition matrix, narrow
[[8. 1. 1. 3. 3.]
 [4. 7. 7. 4. 2.]
 [2. 6. 8. 3. 3.]
 [5. 2. 4. 4. 4.]
 [4. 3. 3. 3. 5.]]
100.0

%s of transitions narrow
bm->bm  8%
bm->br  1%
bm

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]